# Youdin & Shu 2002 Test

We test the advection algorithm by running a fixed particle size drift in a power-law gas surface density / temperature disk. We see how the explicit advection has very little numerical diffusion, while the implicit donor cell scheme is quite diffusive. This is not to say that it cannot be used for `twopoppy2`, because we anyway have real diffusion due to turbulent mixing which is usually stronger.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import twopoppy2
%matplotlib inline

au    = twopoppy2.constants.au
year  = twopoppy2.constants.year

Set up a grid that we use for all simulations

In [ ]:
ri   = np.logspace(-1, 3, 500) * au
grid = twopoppy2.Grid(ri)

Here, we set up a model that we force to behave like a single particle size model. This is done to test the advection algorithm. We broadly follow the setup of Youdin & Shu 2002, Figure 5.

In [ ]:
def testmodel():

    m    = twopoppy2.Twopoppy(grid=grid)

    d2g = 0.01

    m.sigma_g = 1700 * (m.r / au)**-1.5
    m.sigma_g[m.r>200 * au] = 1e-100
    m.sigma_d = d2g * m.sigma_g
    m.T_gas   = m.T_star * 0.05**0.25 * (m.r / m.R_star)**-0.5

    m.initialize()

    m.stokesregime = 0
    m.rho_s = 3.0
    
    # set large grain size to 1 mm
    
    m.a_1 = 0.1

    # set all mass in largest grains
    m.f_mf = 1.0
    m.f_md = 1.0
    
    # we turn off alpha
    
    m.set_all_alpha(1e-100)
    m.update_diffusivity()
    m.update_diffusivity_i()
    
    # we let the gas evolve for 1 year to smooth off any overly sharp edges
    
    m.evolve_gas(year)
    
    # calculate the velocities just once as they will be kept constant
    
    m.update_v_bar()
    m.update_v_bar_i()
    
    return m

### Explicit / Implicit

In [ ]:
# iterate over the snapshots

m_exp = testmodel()
m_imp = testmodel()
sig_d_ini = m_exp.sigma_d.copy()

# set up figure

f, axs = plt.subplots(1, 2, figsize=(12, 6))
for ax in axs:
    ax.set_xlim(.4, 3e2)
    ax.set_ylim(1e-3, 1e3)
    ax.set_xlabel('radius [au]')
    ax.set_ylabel(r'$\Sigma_\mathrm{dust}$ [g / cm$^2$]')
    
# set snapshot times

times = np.array([0, 1e4, 3e4, 7e4, ]) * year
start_index = np.where(m_exp.time >= times)[0][0]

for i_snap in range(start_index, len(times)):
    
    t_next = times[i_snap]
    
    # implicit model
    
    while m_imp.time < t_next:
        dt = min(max(m_imp.time/200, year), t_next - m_imp.time)
        
        m_imp.sigma_d = m_imp._dust_step_impl(dt)[0]
        m_imp.time += dt
        
    # explicit model
        
    while m_exp.time < t_next:
        sig_d, dt = m_exp._dust_step_implexpl(t_max=t_next)
        m_exp.time += dt
        m_exp.sigma_d = sig_d
    
    for m, ax in zip([m_imp, m_exp], axs):
        ax.loglog(m.r / au, m.sigma_d, f'C{i_snap + 1}', label=f't = {m.time / year:.3g} yr')
    
    print(f'\rRunning ... {i_snap/(len(times) - 1) * 100:.1f}%', end='', flush=True)

print('\r------ DONE! ------')

# get the velocity parameters
# the velocity in the analytical solution is defined as
# A * r**d, where A is negative for inward drift.

i_ref = 3
d = m._grid.dlnxdlnr(-m.v_bar)[i_ref]
A = m.v_bar[i_ref] / m.r[i_ref]**d

# overplot the analytical solution

for i, t in enumerate(times):
    for ax in axs:
        ax.loglog(m_exp.r / au, twopoppy2.utils.solution_youdinshu2002(m_exp.r, sig_d_ini, t, A, d), c=f'C{i + 1}', ls='--')

# set legend and title
        
axs[0].plot([0], [0], 'k-', label='numerical')
axs[0].plot([0], [0], 'k--', label='analytical')
axs[0].legend(fontsize='small')

axs[0].set_title('implicit')
axs[1].set_title('explicit');